In [9]:
import os
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
import gymnasium as gym
from gymnasium import spaces
from datetime import datetime
import glob
import torchOptics.optics as tt
import torch.nn as nn
import torchOptics.metrics as tm
import torch.nn.functional as F

# CuDNN 비활성화
torch.backends.cudnn.enabled = False

# 현재 날짜와 시간을 가져와 포맷 지정
current_date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# 간단한 데이터셋 클래스
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, target_dir, meta, transform=None, isTrain=True, padding=0):
        self.target_list = sorted(glob.glob(target_dir + '*.png'))
        self.meta = meta
        self.transform = transform
        self.isTrain = isTrain
        self.padding = padding
        self.random_crop = transforms.RandomCrop((1024, 1024))
        self.center_crop = transforms.CenterCrop(1024)

    def __len__(self):
        return len(self.target_list)

    def __getitem__(self, idx):
        img = tt.imread(self.target_list[idx], meta=self.meta, gray=True).unsqueeze(0)
        if img.shape[-2] < 1024 or img.shape[-1] < 1024:
            img = transforms.Resize((1024, 1024))(img)
        img = self.random_crop(img) if self.isTrain else self.center_crop(img)
        img = transforms.functional.pad(img, (self.padding,) * 4)
        if self.transform:
            img = self.transform(img)
        return img

# BinaryHologramEnv 클래스
class BinaryHologramEnv(gym.Env):
    def __init__(self, target_function, trainloader, max_steps=1000000, T_PSNR=30, T_steps=100):
        super(BinaryHologramEnv, self).__init__()
        # 관찰 공간: (1, 8, 1024, 1024)
        self.observation_space = spaces.Box(low=0, high=1, shape=(1, 8, 1024, 1024), dtype=np.float32)
        # 행동 공간: MultiBinary 데이터
        self.action_space = spaces.MultiBinary(1 * 8 * 1024 * 1024)
        self.target_function = target_function
        self.trainloader = trainloader
        self.max_steps = max_steps
        self.T_PSNR = T_PSNR
        self.T_steps = T_steps
        self.data_iter = iter(self.trainloader)

    def reset(self, seed=None, options=None):
        """
        환경 초기화
        """
        try:
            self.target_image = next(self.data_iter)
        except StopIteration:
            self.data_iter = iter(self.trainloader)
            self.target_image = next(self.data_iter)

        self.target_image = self.target_image.cuda()
        with torch.no_grad():
            self.observation = self.target_function(self.target_image).cpu().numpy().reshape(1, 8, 1024, 1024)

        self.steps = 0
        self.psnr_sustained_steps = 0

        # 튜플 형태로 반환
        return self.observation, {}

    def step(self, action, lr=1e-4, z=2e-3):
        """
        환경의 한 단계 진행
        """
        # Action 데이터를 torch.Tensor로 변환하고 meta 속성 추가
        binary = torch.tensor(action, dtype=torch.float32).cuda()
        binary.meta = {'psize': (7.56e-6, 7.56e-6)}  # 적절한 메타 정보 추가

        # 시뮬레이션 계산
        sim = tt.simulate(binary, z).abs()**2
        result = torch.mean(sim, dim=1, keepdim=True)
        mse = tt.relativeLoss(result, self.target_image, F.mse_loss).detach().cpu().numpy()
        psnr = tt.relativeLoss(result, self.target_image, tm.get_PSNR)
        reward = -mse

        # 상태 업데이트
        terminated = False
        truncated = self.steps >= self.max_steps
        self.steps += 1
        if psnr >= self.T_PSNR:
            self.psnr_sustained_steps += 1
        else:
            self.psnr_sustained_steps = 0
        if self.psnr_sustained_steps >= self.T_steps:
            terminated = True

        # 정보 반환
        info = {"mse": mse, "psnr": psnr, "psnr_sustained_steps": self.psnr_sustained_steps}
        return self.observation, reward, terminated, truncated, info

# 데이터 로더 생성
target_dir = '/nfs/dataset/DIV2K/DIV2K_train_HR/DIV2K_train_HR/'
valid_dir = '/nfs/dataset/DIV2K/DIV2K_valid_HR/DIV2K_valid_HR/'
meta = {'wl': 515e-9, 'dx': (7.56e-6, 7.56e-6)}
padding = 0
train_dataset = SimpleDataset(target_dir=target_dir, meta=meta, isTrain=True, padding=padding)
valid_dataset = SimpleDataset(target_dir=valid_dir, meta=meta, isTrain=False, padding=padding)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False)

# BinaryNet 모델 로드
model = BinaryNet(num_hologram=8, in_planes=1, convReLU=False, convBN=False,
                  poolReLU=False, poolBN=False, deconvReLU=False, deconvBN=False).cuda()
model.load_state_dict(torch.load('result/2024-12-07 19:38:09.105795_pre_reinforce_8_0.002/2024-12-07 19:38:09.105795_pre_reinforce_8_0.002'))
model.eval()

# Gym 환경 생성
env = BinaryHologramEnv(target_function=model, trainloader=train_loader, max_steps=1000, T_PSNR=30, T_steps=100)

# Vectorized 환경 생성
venv = make_vec_env(lambda: env, n_envs=1)
venv = VecNormalize(venv, norm_obs=True, norm_reward=True, clip_obs=10.0)

# PPO 모델 설정 및 학습
ppo_model = PPO(
    policy="MlpPolicy",  # 기본 MLP 정책
    env=venv,
    verbose=1,
    n_steps=128,
    batch_size=64,
    gamma=0.99,
    learning_rate=3e-4,
    tensorboard_log="./ppo_mlp/",
)
ppo_model.learn(total_timesteps=100000)

# 학습된 모델 저장
ppo_model.save(f"ppo_binary_hologram_{current_date}")


NameError: name 'BinaryNet' is not defined

In [10]:
class BinaryNet(nn.Module):
    def __init__(self, num_hologram, final='Sigmoid', in_planes=3,
                 channels=[32, 64, 128, 256, 512, 1024, 2048, 4096],
                 convReLU=True, convBN=True, poolReLU=True, poolBN=True,
                 deconvReLU=True, deconvBN=True):
        super(BinaryNet, self).__init__()

        def CRB2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True, relu=True, bn=True):
            layers = []
            layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                 kernel_size=kernel_size, stride=stride, padding=padding,
                                 bias=bias)]
            if relu:
                layers += [nn.Tanh()]
            if bn:
                layers += [nn.BatchNorm2d(num_features=out_channels)]

            cbr = nn.Sequential(*layers) # *으로 list unpacking 

            return cbr

        def TRB2d(in_channels, out_channels, kernel_size=2, stride=2, bias=True, relu=True, bn=True):
            layers = []
            layers += [nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels,
                                          kernel_size=2, stride=2, padding=0,
                                          bias=True)]
            if bn:
                layers += [nn.BatchNorm2d(num_features=out_channels)]
            if relu:
                layers += [nn.ReLU()]

            cbr = nn.Sequential(*layers) # *으로 list unpacking 

            return cbr

        self.enc1_1 = CRB2d(in_planes, channels[0], relu=convReLU, bn=convBN)
        self.enc1_2 = CRB2d(channels[0], channels[0], relu=convReLU, bn=convBN)
        self.pool1 = CRB2d(channels[0], channels[0], stride=2, relu=poolReLU, bn=poolBN)

        self.enc2_1 = CRB2d(channels[0], channels[1], relu=convReLU, bn=convBN)
        self.enc2_2 = CRB2d(channels[1], channels[1], relu=convReLU, bn=convBN)
        self.pool2 = CRB2d(channels[1], channels[1], stride=2, relu=poolReLU, bn=poolBN)

        self.enc3_1 = CRB2d(channels[1], channels[2], relu=convReLU, bn=convBN)
        self.enc3_2 = CRB2d(channels[2], channels[2], relu=convReLU, bn=convBN)
        self.pool3 = CRB2d(channels[2], channels[2], stride=2, relu=poolReLU, bn=poolBN)

        self.enc4_1 = CRB2d(channels[2], channels[3], relu=convReLU, bn=convBN)
        self.enc4_2 = CRB2d(channels[3], channels[3], relu=convReLU, bn=convBN)
        self.pool4 = CRB2d(channels[3], channels[3], stride=2, relu=poolReLU, bn=poolBN)

        self.enc5_1 = CRB2d(channels[3], channels[4], relu=convReLU, bn=convBN)
        self.enc5_2 = CRB2d(channels[4], channels[4], relu=convReLU, bn=convBN)

        self.deconv4 = TRB2d(channels[4], channels[3], relu=deconvReLU, bn=deconvBN, stride=2)
        self.dec4_1 = CRB2d(channels[4], channels[3], relu=convReLU, bn=convBN)
        self.dec4_2 = CRB2d(channels[3], channels[3], relu=convReLU, bn=convBN)

        self.deconv3 = TRB2d(channels[3], channels[2], relu=deconvReLU, bn=deconvBN, stride=2)
        self.dec3_1 = CRB2d(channels[3], channels[2], relu=convReLU, bn=convBN)
        self.dec3_2 = CRB2d(channels[2], channels[2], relu=convReLU, bn=convBN)

        self.deconv2 = TRB2d(channels[2], channels[1], relu=deconvReLU, bn=deconvBN, stride=2)
        self.dec2_1 = CRB2d(channels[2], channels[1], relu=convReLU, bn=convBN)
        self.dec2_2 = CRB2d(channels[1], channels[1], relu=convReLU, bn=convBN)

        self.deconv1 = TRB2d(channels[1], channels[0], relu=deconvReLU, bn=deconvBN, stride=2)
        self.dec1_1 = CRB2d(channels[1], channels[0], relu=convReLU, bn=convBN)
        self.dec1_2 = CRB2d(channels[0], channels[0], relu=convReLU, bn=convBN)

        self.classifier = CRB2d(channels[0], num_hologram, relu=False, bn=False)

    def forward(self, x):
        # Encoder
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        pool1 = self.pool1(enc1_2)

        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2 = self.pool2(enc2_2)

        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3 = self.pool3(enc3_2)

        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4 = self.pool4(enc4_2)

        enc5_1 = self.enc5_1(pool4)
        enc5_2 = self.enc5_2(enc5_1)

        deconv4 = self.deconv4(enc5_2)
        concat4 = torch.cat((deconv4, enc4_2), dim=1)
        dec4_1 = self.dec4_1(concat4)
        dec4_2 = self.dec4_2(dec4_1)

        deconv3 = self.deconv3(dec4_2)
        concat3 = torch.cat((deconv3, enc3_2), dim=1)
        dec3_1 = self.dec3_1(concat3)
        dec3_2 = self.dec3_2(dec3_1)

        deconv2 = self.deconv2(dec3_2)
        concat2 = torch.cat((deconv2, enc2_2), dim=1)
        dec2_1 = self.dec2_1(concat2)
        dec2_2 = self.dec2_2(dec2_1)

        deconv1 = self.deconv1(dec2_2)
        concat1 = torch.cat((deconv1, enc1_2), dim=1)
        dec1_1 = self.dec1_1(concat1)
        dec1_2 = self.dec1_2(dec1_1)

        # Final classifier
        out = self.classifier(dec1_2)
        out = nn.Sigmoid()(out)
        return out


model = BinaryNet(num_hologram=8, in_planes=1, convReLU=False,
                  convBN=False, poolReLU=False, poolBN=False,
                  deconvReLU=False, deconvBN=False).cuda()
test = torch.randn(1, 1, 1024, 1024).cuda()
out = model(test)
print(out.shape)

torch.Size([1, 8, 1024, 1024])


In [11]:
def rgb_binary_train_padding(model, num_hologram=24, custom_name='0703Unet', epochs=500, lr=1e-4, z=2e-3):
    date = datetime.datetime.now() - datetime.timedelta(hours=15)
    model.apply(initialize_weights)
    mean_psnr_list = []
    mean_mse_list = []
    max_epoch = 0
    max_psnr = 0
    crop = torchvision.transforms.CenterCrop(896)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    sign = SignFunction.apply
    # path = 'models/'
    model_name = f'{date}_{custom_name}_{num_hologram}_{z}'
    # make result folder
    result_folder = 'result/'+model_name
    os.mkdir(result_folder)
    num_parameters = count_parameters(model)
    model_path = os.path.join(result_folder, model_name)
    psnr_result_path = os.path.join(result_folder, 'meanPSNR.npy')
    mse_result_path = os.path.join(result_folder, 'meanMSE.npy')
    result = {'model_path': model_path, 'psnr_result_path': psnr_result_path,
          'mse_result_path': mse_result_path, 'z': z,
          'num_hologram': num_hologram, 'lr': lr, 'epochs': epochs,
            'num_parameters': num_parameters}
    # save result dict
    with open(os.path.join(result_folder, 'result_dict.pickle'), 'wb') as fw:
        pickle.dump(result, fw)
    for epoch in range(epochs):
        pbar = tqdm.tqdm(trainloader)
        model.train()
        for target in pbar:
            target = target
            out = model(target)
            binary = sign(out)
            sim = tt.simulate(binary, 2e-3).abs()**2
            mean_sim = torch.mean(sim, dim=1, keepdim=True)
            rgb = crop(mean_sim)
            croped_target = crop(target)
            loss = tt.relativeLoss(rgb, croped_target, F.mse_loss)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        model.eval()
        with torch.no_grad():
            psnrList = []
            mseList = []
            for idx, valid in enumerate(validloader):
                valid = valid
                out = model(valid)
                binary = sign(out)
                sim = tt.simulate(binary, 2e-3).abs()**2
                mean_sim = torch.mean(sim, dim=1, keepdim=True)
                rgb = crop(mean_sim)
                croped_target = crop(valid)
                psnr = tt.relativeLoss(rgb, croped_target, tm.get_PSNR)
                psnrList.append(psnr)
                mse = tt.relativeLoss(rgb, croped_target, F.mse_loss).detach().cpu().numpy()
                mseList.append(mse)
                if idx == 0:
                    pl = tt.show_with_insets(rgb, croped_target, correct_colorwise=True)
                    pl.save(os.path.join(result_folder, 'penguin.png'))
            mean_psnr = sum(psnrList)/len(psnrList)
            mean_mse = sum(mseList)/len(mseList)
            mean_psnr_list.append(mean_psnr)
            mean_mse_list.append(mean_mse)
            # save psnr and mse
            if epoch % 10 == 0:
                np.save(psnr_result_path, mean_psnr_list)
                np.save(mse_result_path, mean_mse_list)
                plt.plot(np.arange(len(mean_psnr_list)), mean_psnr_list,
                         label=f'max psnr: {max(mean_psnr_list)}')
                plt.title(model_name+'_psnr')
                plt.legend()
                plt.savefig(os.path.join(result_folder, 'meanPSNR.png'))
                plt.clf()
                plt.plot(np.arange(len(mean_mse_list)), mean_mse_list,
                         label=f'max psnr: {min(mean_mse_list)}')
                plt.title(model_name+'_mse')
                plt.legend()
                plt.savefig(os.path.join(result_folder, 'meanMSE.png'))
                plt.clf()
            print(f'mean PSNR : {mean_psnr} {epoch}/{epochs}')
            print(f'mean MSE : {mean_mse} {epoch}/{epochs}')

        if mean_psnr > max_psnr:
            max_psnr = mean_psnr
            max_epoch = epoch
            torch.save(model.state_dict(), model_path)
        print(f'max_psnr: {max_psnr}, epoch: {max_epoch}')
        print(f'Model saved at: {model_path}')

In [12]:
valid_dataset[0].shape

/home/songyb111/torchOptics/optics.py:151: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:294.)
  return super().__new__(cls, torch.Tensor(x), *args, **kwargs).to(device)
/tmp/ipykernel_644575/3672340644.py:39: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:294.)
  img = tt.imread(self.target_list[idx], meta=self.meta, gray=True).unsqueeze(0)
/tmp/ipykernel_644575/3672340644.py:40: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:294.)
  if img.shape[-2] < 1024 or img.shape[-1] < 1024:
/usr/local

torch.Size([1, 1024, 1024])

In [13]:
sign = SignFunction.apply
out = model(valid_dataset[0].unsqueeze(0).unsqueeze(0))
binary = sign(out, 0.5)
sim = tt.simulate(binary, 2e-3)
mean_sim = torch.mean(sim, dim=1, keepdim=True)

NameError: name 'SignFunction' is not defined

In [14]:
for target in trainloader:
    print(target.shape)
    break

NameError: name 'trainloader' is not defined

In [15]:
target

NameError: name 'target' is not defined

In [16]:
mean_sim

NameError: name 'mean_sim' is not defined

In [18]:
import os
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
import gymnasium as gym
from gymnasium import spaces
from datetime import datetime
import glob
import torchOptics.optics as tt
import torch.nn.functional as F
import torchOptics.metrics as tm

# CuDNN 비활성화
torch.backends.cudnn.enabled = False

# 현재 날짜와 시간을 가져와 포맷 지정
current_date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# 간단한 데이터셋 클래스
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, target_dir, meta, transform=None, isTrain=True, padding=0):
        self.target_list = sorted(glob.glob(target_dir + '*.png'))
        self.meta = meta
        self.transform = transform
        self.isTrain = isTrain
        self.padding = padding
        self.random_crop = transforms.RandomCrop((1024, 1024))
        self.center_crop = transforms.CenterCrop(1024)

    def __len__(self):
        return len(self.target_list)

    def __getitem__(self, idx):
        img = tt.imread(self.target_list[idx], meta=self.meta, gray=True).unsqueeze(0)
        if img.shape[-2] < 1024 or img.shape[-1] < 1024:
            img = transforms.Resize((1024, 1024))(img)
        img = self.random_crop(img) if self.isTrain else self.center_crop(img)
        img = transforms.functional.pad(img, (self.padding,) * 4)
        if self.transform:
            img = self.transform(img)
        return img

# 사용자 정의 Tensor 클래스
class CustomTensor(torch.Tensor):
    """
    Torch Tensor에 meta 속성을 추가하는 클래스.
    """
    @staticmethod
    def __new__(cls, tensor, meta):
        obj = torch.Tensor._make_subclass(cls, tensor, require_grad=tensor.requires_grad)
        obj.meta = meta
        return obj

    def __init__(self, tensor, meta):
        self.meta = meta

# BinaryHologramEnv 클래스
class BinaryHologramEnv(gym.Env):
    def __init__(self, target_function, trainloader, meta, max_steps=1000000, T_PSNR=30, T_steps=100):
        super(BinaryHologramEnv, self).__init__()
        # 관찰 공간: (1, 8, 1024, 1024)
        self.observation_space = spaces.Box(low=0, high=1, shape=(1, 8, 1024, 1024), dtype=np.float32)
        # 행동 공간: MultiBinary 데이터
        self.action_space = spaces.MultiBinary(1 * 8 * 1024 * 1024)
        self.target_function = target_function
        self.trainloader = trainloader
        self.meta = meta
        self.max_steps = max_steps
        self.T_PSNR = T_PSNR
        self.T_steps = T_steps
        self.data_iter = iter(self.trainloader)

    def reset(self, seed=None, options=None):
        """
        환경 초기화
        """
        try:
            self.target_image = next(self.data_iter)
        except StopIteration:
            self.data_iter = iter(self.trainloader)
            self.target_image = next(self.data_iter)

        self.target_image = self.target_image.cuda()
        with torch.no_grad():
            self.observation = self.target_function(self.target_image).cpu().numpy().reshape(1, 8, 1024, 1024)

        self.steps = 0
        self.psnr_sustained_steps = 0

        # 튜플 형태로 반환
        return self.observation, {}

    def step(self, action, lr=1e-4, z=2e-3):
        """
        환경의 한 단계 진행
        """
        # Action 데이터를 CustomTensor로 변환하고 meta 속성 추가
        binary = CustomTensor(torch.tensor(action, dtype=torch.float32).cuda(), meta=self.meta)

        # 시뮬레이션 계산
        sim = tt.simulate(binary, z).abs()**2
        result = torch.mean(sim, dim=1, keepdim=True)
        mse = tt.relativeLoss(result, self.target_image, F.mse_loss).detach().cpu().numpy()
        psnr = tt.relativeLoss(result, self.target_image, tm.get_PSNR)
        reward = -mse

        # 상태 업데이트
        terminated = False
        truncated = self.steps >= self.max_steps
        self.steps += 1
        if psnr >= self.T_PSNR:
            self.psnr_sustained_steps += 1
        else:
            self.psnr_sustained_steps = 0
        if self.psnr_sustained_steps >= self.T_steps:
            terminated = True

        # 정보 반환
        info = {"mse": mse, "psnr": psnr, "psnr_sustained_steps": self.psnr_sustained_steps}
        return self.observation, reward, terminated, truncated, info

# 데이터 로더 생성
target_dir = '/nfs/dataset/DIV2K/DIV2K_train_HR/DIV2K_train_HR/'
valid_dir = '/nfs/dataset/DIV2K/DIV2K_valid_HR/DIV2K_valid_HR/'
meta = {'wl': 515e-9, 'dx': (7.56e-6, 7.56e-6)}
padding = 0
train_dataset = SimpleDataset(target_dir=target_dir, meta=meta, isTrain=True, padding=padding)
valid_dataset = SimpleDataset(target_dir=valid_dir, meta=meta, isTrain=False, padding=padding)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False)

# BinaryNet 모델 로드
model = BinaryNet(num_hologram=8, in_planes=1, convReLU=False, convBN=False,
                  poolReLU=False, poolBN=False, deconvReLU=False, deconvBN=False).cuda()
model.load_state_dict(torch.load('result/2024-12-07 19:38:09.105795_pre_reinforce_8_0.002/2024-12-07 19:38:09.105795_pre_reinforce_8_0.002'))
model.eval()

# Gym 환경 생성
env = BinaryHologramEnv(target_function=model, trainloader=train_loader, meta=meta, max_steps=1000, T_PSNR=30, T_steps=100)

# Vectorized 환경 생성
venv = make_vec_env(lambda: env, n_envs=1)
venv = VecNormalize(venv, norm_obs=True, norm_reward=True, clip_obs=10.0)

# PPO 모델 설정 및 학습
ppo_model = PPO(
    policy="MlpPolicy",  # 기본 MLP 정책
    env=venv,
    verbose=1,
    n_steps=128,
    batch_size=64,
    gamma=0.99,
    learning_rate=3e-4,
    tensorboard_log="./ppo_mlp/",
)
ppo_model.learn(total_timesteps=100000)

# 학습된 모델 저장
ppo_model.save(f"ppo_binary_hologram_{current_date}")


/tmp/ipykernel_644575/2733682670.py:140: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('result/2024-12-07 19:38:09.105795_pre_reinforce_8_0.

Using cuda device


/tmp/ipykernel_644575/2733682670.py:38: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:294.)
  img = tt.imread(self.target_list[idx], meta=self.meta, gray=True).unsqueeze(0)
/tmp/ipykernel_644575/2733682670.py:39: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:294.)
  if img.shape[-2] < 1024 or img.shape[-1] < 1024:
/tmp/ipykernel_644575/2733682670.py:87: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:294.)
  self.target_image = self.target_image.cuda()
/tmp/ipykernel_644575/3687238895.py:95: U

Logging to ./ppo_mlp/PPO_4


AttributeError: 'CustomTensor' object has no attribute 'meta'

In [ ]:
from stable_baselines3.common.env_checker import check_env

def target_function(state, action):
    """
    타겟 함수: 상태와 행동 간의 MSE를 계산합니다.
    """
    target = state.mean(axis=-1, keepdims=True)
    return np.mean((action.mean(axis=-1) - target.squeeze()) ** 2)

# 환경 인스턴스 생성
env = BinaryHologramEnv(
    target_function=target_function,
    max_steps=1000000,
    T_PSNR=30,
    T_steps=100,
)

# 환경 유효성 확인
check_env(env, warn=True)
